In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [106]:
def dydx(x,y):
    #set the derivatives
    #our equation is d^2x/dx^2 = -y
    #so we can write
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0]
    #we will set z = y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #return arrays of dydx
    return y_derivs

In [107]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    
    #declare k? arrays (? any digit 0-9)
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    #define coefficients
    a21 = 1/5
    a31 = 3/40
    a41 = 3/10
    a51 = (-11)/54
    a61 = 1631/55296
    
    a32 = 9/40
    a42 = (-9)/10
    a52 = 5/2
    a62 = 175/512
    
    a43 = 6/5
    a53 = (-70)/27
    a63 =575/13824
    
    a54 = 35/27
    a64 = 44275/110592
    
    a65 = 253/4096
    
    a2 = 1/5
    a3 = 3/10
    a4 = 3/5
    a5 = 1
    a6 = 7/8
    
    c1 = 37/378
    c2 = 0
    c3 = 250/621
    c4 = 125/594
    c5 = 0
    c6 = 512/1771
    
    c1_2 = 2825/27648
    c2_2 = 0
    c3_2 = 18575/48384
    c4_2 = 12525/55296
    c5_2 = 277/14336
    c6_2 = 1/4
    
    #define x step
    x_ipoh_k2 = xi + a2*h
    x_ipoh_k3 = xi + a3*h
    x_ipoh_k4 = xi + a4*h
    x_ipoh_k5 = xi + a5*h
    x_ipoh_k6 = xi + a6*h
    
    #define y step
    x_ipo_k2 =  a21*k1[:]
    x_ipo_k3 =  a31*k1[:] + a32*k2[:]
    x_ipo_k4 =  a41*k1[:] + a42*k2[:] + a43*k3[:]
    x_ipo_k5 =  a51*k1[:] + a52*k2[:] + a53*k3[:] + a54*k4[:]
    x_ipo_k6 =  a61*k1[:] + a62*k2[:] + a63*k3[:] + a64*k4[:] + a65*k5[:]
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get the k2 values
    y_temp[:] = yi[:] + x_ipo_k2
    y_derivs = dydx(x_ipoh_k2,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + x_ipo_k3
    y_derivs = dydx(x_ipoh_k3,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + x_ipo_k4
    y_derivs = dydx(x_ipoh_k4,y_temp)
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    y_temp[:] = yi[:] + x_ipo_k5
    y_derivs = dydx(x_ipoh_k5,y_temp)
    k5[:] = h*y_derivs[:]
    
    #get k6 values
    y_temp[:] = yi[:] + x_ipo_k6
    y_derivs = dydx(x_ipoh_k6,y_temp)
    k6[:] = h*y_derivs[:]
    
    #advance y by a step h
    yipo = yi + c1*k1 + c2*k2 + c3*k3 + c4*k4 + c5*k5 + c6*k6 
    
    #error step
    yipo2 = yi + c1_2*k1 + c2_2*k2 + c3_2*k3 + c4_2*k4 + c5_2*k5 + c6_2*k6 
    
    #this is an array (yipo = y i plus one)
    return yipo, yipo2

In [108]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterarions
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust the step
    while(Delta.max()/tol > 1.0):
        
        yipo, yipo2 =  rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        
        e1 = yipo
        e2 = yipo2
        
        #compute an error
        Delta = np.fabs(e1-e2)
        
        #if the error is too large, decrease the step
        if (Delta.max()/tol > 1.0):
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
           
        #check iterations
        if (i>=imax): 
            print("too many iterations in rk4_mv_ad()")
            raise StopIteration("ending after i =",i)
            
        #iterate
        i+= 1
        
    #next time try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return yipo, h_new, h_step

In [109]:
def rk4_mv(dydx,a,b,y_a,tol):
    
    #dydx is the derivative wrt x
    #a is lower bound and b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    
    #an intial step size == very small
    h = 1.0e-4 * (b-a)
    
    #set number 0f iterations
    imax = 10000
   
    #set  an iteration variable
    i = 0
    
    #set the number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set the numper of coupled odes to the size of y_a
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set initial conditions
    flag = 1
    
    #loop until we reach th right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if (xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            
            flag = 0
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        #prevent too many interations
        if(i>=imax):
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number =",i)
            
        #iterate
        i += 1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i, xi, h_step, b)
        print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = 0
            
    return x,y

In [110]:
a = 0.0
b = 2.0*np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform the integration
x,y = rk4_mv(dydx,a,b,y_0,tolerance)

i =   1	x = 0.00005309	h = 0.00005309	b=6.28318531
i =   2	x = 0.00010816	h = 0.00005507	b=6.28318531
i =   3	x = 0.00016343	h = 0.00005527	b=6.28318531
i =   4	x = 0.00021872	h = 0.00005529	b=6.28318531
i =   5	x = 0.00027402	h = 0.00005530	b=6.28318531
i =   6	x = 0.00032932	h = 0.00005530	b=6.28318531
i =   7	x = 0.00038461	h = 0.00005530	b=6.28318531
i =   8	x = 0.00043991	h = 0.00005530	b=6.28318531
i =   9	x = 0.00049520	h = 0.00005530	b=6.28318531
i =  10	x = 0.00055050	h = 0.00005530	b=6.28318531
i =  11	x = 0.00060580	h = 0.00005530	b=6.28318531
i =  12	x = 0.00066109	h = 0.00005530	b=6.28318531
i =  13	x = 0.00071639	h = 0.00005530	b=6.28318531
i =  14	x = 0.00077168	h = 0.00005530	b=6.28318531
i =  15	x = 0.00082698	h = 0.00005530	b=6.28318531
i =  16	x = 0.00088228	h = 0.00005530	b=6.28318531
i =  17	x = 0.00093757	h = 0.00005530	b=6.28318531
i =  18	x = 0.00099287	h = 0.00005530	b=6.28318531
i =  19	x = 0.00104816	h = 0.00005530	b=6.28318531
i =  20	x = 0.00110346	h = 0.00

i = 733	x = 0.04054052	h = 0.00005534	b=6.28318531
i = 734	x = 0.04059586	h = 0.00005534	b=6.28318531
i = 735	x = 0.04065120	h = 0.00005534	b=6.28318531
i = 736	x = 0.04070654	h = 0.00005534	b=6.28318531
i = 737	x = 0.04076188	h = 0.00005534	b=6.28318531
i = 738	x = 0.04081722	h = 0.00005534	b=6.28318531
i = 739	x = 0.04087257	h = 0.00005534	b=6.28318531
i = 740	x = 0.04092791	h = 0.00005534	b=6.28318531
i = 741	x = 0.04098325	h = 0.00005534	b=6.28318531
i = 742	x = 0.04103859	h = 0.00005534	b=6.28318531
i = 743	x = 0.04109394	h = 0.00005534	b=6.28318531
i = 744	x = 0.04114928	h = 0.00005534	b=6.28318531
i = 745	x = 0.04120462	h = 0.00005534	b=6.28318531
i = 746	x = 0.04125996	h = 0.00005534	b=6.28318531
i = 747	x = 0.04131531	h = 0.00005534	b=6.28318531
i = 748	x = 0.04137065	h = 0.00005534	b=6.28318531
i = 749	x = 0.04142599	h = 0.00005534	b=6.28318531
i = 750	x = 0.04148134	h = 0.00005534	b=6.28318531
i = 751	x = 0.04153668	h = 0.00005534	b=6.28318531
i = 752	x = 0.04159202	h = 0.00

i = 1317	x = 0.07288658	h = 0.00005544	b=6.28318531
i = 1318	x = 0.07294202	h = 0.00005544	b=6.28318531
i = 1319	x = 0.07299747	h = 0.00005544	b=6.28318531
i = 1320	x = 0.07305291	h = 0.00005544	b=6.28318531
i = 1321	x = 0.07310835	h = 0.00005544	b=6.28318531
i = 1322	x = 0.07316380	h = 0.00005544	b=6.28318531
i = 1323	x = 0.07321924	h = 0.00005544	b=6.28318531
i = 1324	x = 0.07327468	h = 0.00005544	b=6.28318531
i = 1325	x = 0.07333013	h = 0.00005544	b=6.28318531
i = 1326	x = 0.07338557	h = 0.00005544	b=6.28318531
i = 1327	x = 0.07344102	h = 0.00005544	b=6.28318531
i = 1328	x = 0.07349646	h = 0.00005545	b=6.28318531
i = 1329	x = 0.07355191	h = 0.00005545	b=6.28318531
i = 1330	x = 0.07360735	h = 0.00005545	b=6.28318531
i = 1331	x = 0.07366280	h = 0.00005545	b=6.28318531
i = 1332	x = 0.07371825	h = 0.00005545	b=6.28318531
i = 1333	x = 0.07377369	h = 0.00005545	b=6.28318531
i = 1334	x = 0.07382914	h = 0.00005545	b=6.28318531
i = 1335	x = 0.07388459	h = 0.00005545	b=6.28318531
i = 1336	x =

i = 1889	x = 0.10464191	h = 0.00005560	b=6.28318531
i = 1890	x = 0.10469751	h = 0.00005560	b=6.28318531
i = 1891	x = 0.10475311	h = 0.00005560	b=6.28318531
i = 1892	x = 0.10480871	h = 0.00005560	b=6.28318531
i = 1893	x = 0.10486431	h = 0.00005560	b=6.28318531
i = 1894	x = 0.10491991	h = 0.00005560	b=6.28318531
i = 1895	x = 0.10497551	h = 0.00005560	b=6.28318531
i = 1896	x = 0.10503112	h = 0.00005560	b=6.28318531
i = 1897	x = 0.10508672	h = 0.00005560	b=6.28318531
i = 1898	x = 0.10514232	h = 0.00005560	b=6.28318531
i = 1899	x = 0.10519792	h = 0.00005560	b=6.28318531
i = 1900	x = 0.10525352	h = 0.00005560	b=6.28318531
i = 1901	x = 0.10530913	h = 0.00005560	b=6.28318531
i = 1902	x = 0.10536473	h = 0.00005560	b=6.28318531
i = 1903	x = 0.10542034	h = 0.00005560	b=6.28318531
i = 1904	x = 0.10547594	h = 0.00005560	b=6.28318531
i = 1905	x = 0.10553154	h = 0.00005560	b=6.28318531
i = 1906	x = 0.10558715	h = 0.00005560	b=6.28318531
i = 1907	x = 0.10564275	h = 0.00005561	b=6.28318531
i = 1908	x =

i = 2417	x = 0.13404801	h = 0.00005580	b=6.28318531
i = 2418	x = 0.13410381	h = 0.00005580	b=6.28318531
i = 2419	x = 0.13415960	h = 0.00005580	b=6.28318531
i = 2420	x = 0.13421540	h = 0.00005580	b=6.28318531
i = 2421	x = 0.13427120	h = 0.00005580	b=6.28318531
i = 2422	x = 0.13432699	h = 0.00005580	b=6.28318531
i = 2423	x = 0.13438279	h = 0.00005580	b=6.28318531
i = 2424	x = 0.13443859	h = 0.00005580	b=6.28318531
i = 2425	x = 0.13449439	h = 0.00005580	b=6.28318531
i = 2426	x = 0.13455019	h = 0.00005580	b=6.28318531
i = 2427	x = 0.13460599	h = 0.00005580	b=6.28318531
i = 2428	x = 0.13466179	h = 0.00005580	b=6.28318531
i = 2429	x = 0.13471759	h = 0.00005580	b=6.28318531
i = 2430	x = 0.13477339	h = 0.00005580	b=6.28318531
i = 2431	x = 0.13482919	h = 0.00005580	b=6.28318531
i = 2432	x = 0.13488499	h = 0.00005580	b=6.28318531
i = 2433	x = 0.13494080	h = 0.00005580	b=6.28318531
i = 2434	x = 0.13499660	h = 0.00005580	b=6.28318531
i = 2435	x = 0.13505240	h = 0.00005580	b=6.28318531
i = 2436	x =

...

i = 8482	x = 0.48816270	h = 0.00006260	b=6.28318531
i = 8483	x = 0.48822531	h = 0.00006261	b=6.28318531
i = 8484	x = 0.48828792	h = 0.00006261	b=6.28318531
i = 8485	x = 0.48835053	h = 0.00006261	b=6.28318531
i = 8486	x = 0.48841314	h = 0.00006261	b=6.28318531
i = 8487	x = 0.48847575	h = 0.00006261	b=6.28318531
i = 8488	x = 0.48853837	h = 0.00006262	b=6.28318531
i = 8489	x = 0.48860099	h = 0.00006262	b=6.28318531
i = 8490	x = 0.48866361	h = 0.00006262	b=6.28318531
i = 8491	x = 0.48872623	h = 0.00006262	b=6.28318531
i = 8492	x = 0.48878886	h = 0.00006262	b=6.28318531
i = 8493	x = 0.48885148	h = 0.00006263	b=6.28318531
i = 8494	x = 0.48891411	h = 0.00006263	b=6.28318531
i = 8495	x = 0.48897674	h = 0.00006263	b=6.28318531
i = 8496	x = 0.48903937	h = 0.00006263	b=6.28318531
i = 8497	x = 0.48910201	h = 0.00006263	b=6.28318531
i = 8498	x = 0.48916465	h = 0.00006264	b=6.28318531
i = 8499	x = 0.48922729	h = 0.00006264	b=6.28318531
i = 8500	x = 0.48928993	h = 0.00006264	b=6.28318531
i = 8501	x =

i = 8948	x = 0.51757027	h = 0.00006362	b=6.28318531
i = 8949	x = 0.51763390	h = 0.00006363	b=6.28318531
i = 8950	x = 0.51769753	h = 0.00006363	b=6.28318531
i = 8951	x = 0.51776116	h = 0.00006363	b=6.28318531
i = 8952	x = 0.51782479	h = 0.00006363	b=6.28318531
i = 8953	x = 0.51788843	h = 0.00006364	b=6.28318531
i = 8954	x = 0.51795207	h = 0.00006364	b=6.28318531
i = 8955	x = 0.51801571	h = 0.00006364	b=6.28318531
i = 8956	x = 0.51807935	h = 0.00006364	b=6.28318531
i = 8957	x = 0.51814299	h = 0.00006364	b=6.28318531
i = 8958	x = 0.51820664	h = 0.00006365	b=6.28318531
i = 8959	x = 0.51827029	h = 0.00006365	b=6.28318531
i = 8960	x = 0.51833394	h = 0.00006365	b=6.28318531
i = 8961	x = 0.51839760	h = 0.00006365	b=6.28318531
i = 8962	x = 0.51846125	h = 0.00006366	b=6.28318531
i = 8963	x = 0.51852491	h = 0.00006366	b=6.28318531
i = 8964	x = 0.51858857	h = 0.00006366	b=6.28318531
i = 8965	x = 0.51865224	h = 0.00006366	b=6.28318531
i = 8966	x = 0.51871590	h = 0.00006367	b=6.28318531
i = 8967	x =

i = 9405	x = 0.54689597	h = 0.00006473	b=6.28318531
i = 9406	x = 0.54696070	h = 0.00006473	b=6.28318531
i = 9407	x = 0.54702544	h = 0.00006474	b=6.28318531
i = 9408	x = 0.54709018	h = 0.00006474	b=6.28318531
i = 9409	x = 0.54715492	h = 0.00006474	b=6.28318531
i = 9410	x = 0.54721967	h = 0.00006474	b=6.28318531
i = 9411	x = 0.54728441	h = 0.00006475	b=6.28318531
i = 9412	x = 0.54734916	h = 0.00006475	b=6.28318531
i = 9413	x = 0.54741392	h = 0.00006475	b=6.28318531
i = 9414	x = 0.54747867	h = 0.00006476	b=6.28318531
i = 9415	x = 0.54754343	h = 0.00006476	b=6.28318531
i = 9416	x = 0.54760819	h = 0.00006476	b=6.28318531
i = 9417	x = 0.54767295	h = 0.00006476	b=6.28318531
i = 9418	x = 0.54773772	h = 0.00006477	b=6.28318531
i = 9419	x = 0.54780249	h = 0.00006477	b=6.28318531
i = 9420	x = 0.54786726	h = 0.00006477	b=6.28318531
i = 9421	x = 0.54793203	h = 0.00006477	b=6.28318531
i = 9422	x = 0.54799681	h = 0.00006478	b=6.28318531
i = 9423	x = 0.54806158	h = 0.00006478	b=6.28318531
i = 9424	x =

i = 9859	x = 0.57655701	h = 0.00006595	b=6.28318531
i = 9860	x = 0.57662296	h = 0.00006595	b=6.28318531
i = 9861	x = 0.57668892	h = 0.00006596	b=6.28318531
i = 9862	x = 0.57675488	h = 0.00006596	b=6.28318531
i = 9863	x = 0.57682084	h = 0.00006596	b=6.28318531
i = 9864	x = 0.57688681	h = 0.00006597	b=6.28318531
i = 9865	x = 0.57695278	h = 0.00006597	b=6.28318531
i = 9866	x = 0.57701875	h = 0.00006597	b=6.28318531
i = 9867	x = 0.57708472	h = 0.00006597	b=6.28318531
i = 9868	x = 0.57715070	h = 0.00006598	b=6.28318531
i = 9869	x = 0.57721668	h = 0.00006598	b=6.28318531
i = 9870	x = 0.57728266	h = 0.00006598	b=6.28318531
i = 9871	x = 0.57734865	h = 0.00006599	b=6.28318531
i = 9872	x = 0.57741464	h = 0.00006599	b=6.28318531
i = 9873	x = 0.57748063	h = 0.00006599	b=6.28318531
i = 9874	x = 0.57754662	h = 0.00006599	b=6.28318531
i = 9875	x = 0.57761262	h = 0.00006600	b=6.28318531
i = 9876	x = 0.57767862	h = 0.00006600	b=6.28318531
i = 9877	x = 0.57774462	h = 0.00006600	b=6.28318531
i = 9878	x =

StopIteration: ('Iteration number =', 10000)

In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)

In [ ]:
sin = np.sin(x)
cosin = np.cos(x)

y_error = (y[:,0]-sin)
dydx_error = (y[:,1]-cosin)

plt.plot(x,y_error,label="y(x) Error")
plt.plot(x,dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)